# Controllable Agents for RAG

Adding agentic capabilities on top of your RAG pipeline can allow you to reason over much more complex questions.

But a big pain point for agents is the **lack of steerability/transparency**. An agent may tackle a user query through chain-of-thought/planning, which requires repeated calls to an LLM. During this process it can be hard to inspect what's going on, or stop/correct execution in the middle.

This notebook shows you how to use our brand-new lower-level agent API, which allows controllable step-wise execution, on top of a RAG pipeline.

We showcase this over Wikipedia documents.

In [ ]:
!pip install llama-index

## Setup Data

Here we load a simple dataset of different cities from Wikipedia.

In [ ]:
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.schema import IndexNode
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI

In [ ]:
wiki_titles = [
    "Toronto",
    "Seattle",
    "Chicago",
    "Boston",
    "Houston",
]

In [ ]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            # 'exintro': True,
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [ ]:
# Load all wiki documents
city_docs = {}
for wiki_title in wiki_titles:
    city_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

Define LLM + Service Context + Callback Manager

In [ ]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

## Setup Agent

In this section we define our tools and setup the agent.

### Define Toolset

Each tool here corresponds to a simple top-k RAG pipeline over a single document / Wikipedia page.

In [ ]:
from llama_index.agent import OpenAIAgent
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser import SentenceSplitter
import os

node_parser = SentenceSplitter()

# Build agents dictionary
query_engine_tools = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(city_docs[wiki_title])

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes, service_context=service_context)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
            service_context=service_context,
        )
    # define query engines
    vector_query_engine = vector_index.as_query_engine()

    # define tools
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{wiki_title}",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, arts and culture,"
                    " sports, demographics, or more)."
                ),
            ),
        )
    )

### Setup OpenAI Agent

We setup an OpenAI Agent through its components: an AgentRunner as well as an `OpenAIAgentWorker`.

In [ ]:
from llama_index.agent.executor.base import AgentRunner
from llama_index.agent.openai.step import OpenAIAgentWorker

openai_step_engine = OpenAIAgentWorker.from_tools(
    query_engine_tools, llm=llm, verbose=True
)
agent = AgentRunner(openai_step_engine)

## Run Some Queries

We now demonstrate the capabilities of our step-wise agent framework. 

We show how it can handle complex queries, both e2e as well as step by step. 

We can then show how we can steer the outputs.

### Out of the box

In [ ]:
response = agent.chat(
    "Tell me about the demographics of Houston, and compare that with the demographics of Chicago"
)

In [ ]:
print(str(response))

### Test Step-Wise Execution

We now break this query down into steps. We first create a task object from the user query.

We can then start running through steps - or even interjecting our own.

In [ ]:
# start task
task = agent.create_task(
    "Tell me about the demographics of Houston, and compare that with the demographics of Chicago?"
)

This returns a `Task` object, which contains the `input`, additional state in `extra_state`, and other fields.

Now let's try executing a single step of this task.

In [ ]:
step_output = agent.run_step(task)

In [ ]:
task.extra_state["new_memory"].get_all()

When we inspect the logs and the output, we see that the first part was executed - the demographics of Houston.

If you wanted to pause execution now, you can - you can take the intermediate results without completing the agent flow!

**NOTE**: The `memory` of the agent (`agent.memory`) isn't modified until the task is complete and committed - so if you pause now, the memory won't be committed.

Let's run the next two steps.

In [ ]:
step_output = agent.run_step(task)

In [ ]:
step_output = agent.run_step(task)

Since the steps look good, we are now ready to call `finalize_response`, get back our response.

This will also commit the task execution to the `memory` object present in our `agent_runner`. We can inspect it.

In [ ]:
response = agent.finalize_response(task)

In [ ]:
agent.memory.get_all()

### Test Injecting Intermediate Steps

You can actually choose to inject or edit intermediate steps if you choose. Here we show an example of this in action, we'll edit the query on the fly.

**NOTE**: This may not work for all agent worker types, especially those with complex dependencies between steps. Injecting steps works well for "simpler"/more linear agent loops such as ReAct.

TODO.

### Inspect Steps / Tasks

We can inspect current and previous tasks and steps.

This gives you greater transparency into what the agent has processed!

In [ ]:
tasks = agent.list_tasks()
print(tasks)

In [ ]:
task = tasks[0]
steps = agent.list_completed_steps(task())